In [ ]:
import biospat.biospat as biospat_full
from biospat.stand_alone_functions import (
    merge_touching_groups,
    classify_range_edges,
    assign_polygon_clusters,
    update_polygon_categories,
    process_gbif_csv,
    fetch_gbif_data,
    convert_to_gdf,
    make_dbscan_polygons_with_points_from_gdf,
    get_start_year_from_species,
    prune_by_year,
    clip_polygons_to_continent,
    assign_polygon_clusters_gbif,
    classify_range_edges_gbif,
    update_polygon_categories_gbif,
    merge_and_remap_polygons,
    remove_lakes_and_plot_gbif,
)
from biospat.references_data import REFERENCES

In [ ]:
euc_list = fetch_gbif_data("Acer rubrum", limit=1000)
euc_gdf = convert_to_gdf(euc_list)
polys = make_dbscan_polygons_with_points_from_gdf(euc_gdf)
new_polys = prune_by_year(polys, 1971, 2025)
merged_polygons = merge_and_remap_polygons(new_polys, buffer_distance=10)

unique_polys_no_lakes = remove_lakes_and_plot_gbif(merged_polygons)

clipped_polys = clip_polygons_to_continent(unique_polys_no_lakes)

clipped_polys

In [ ]:
import geopandas as gpd


new_map = biospat_full.Map()

# Load your range polygons and lakes
new_map.load_historic_data("Acer negundo")  # Replace with your actual species name

# Remove lakes from the range polygons
range_no_lakes = new_map.remove_lakes(new_map.gdfs["acernegu"])

merged_polygons = merge_touching_groups(range_no_lakes, buffer_distance=5000)

island_states_gdf = gpd.read_file("../../island_states.gpkg")

# Find and cluster polygons

clustered_polygons, largest_polygons = assign_polygon_clusters(merged_polygons)

# clustered_polygons.plot(column='cluster', figsize=(10, 10))

classified_polygons = classify_range_edges(clustered_polygons, largest_polygons)

updated_polygon = update_polygon_categories(
    largest_polygons, classified_polygons, island_states_gdf
)

updated_polygon.plot(column="category", figsize=(10, 10), legend=True)